In [1]:
import os
import pandas as pd

pd.options.display.float_format = '{:,.0f}'.format

In [2]:
# RESULTS CPA-CHECKER
TRUE = "Verification result: TRUE. No property violation found by chosen configuration."
FALSE = "Verification result: FALSE. Property violation (termination) found by chosen configuration."
UNKNOWN = "Could not synthesize a termination or non-termination argument."
TIMEOUT = "Shutdown requested (The walltime limit of 10s has elapsed.), waiting for termination."

# ERRORS CPA-CHECKER
OPERATION = "Exception in thread \"main\" java.lang.UnsupportedOperationException"
UNSOUND = "Analysis 1 terminated, but result is unsound."
PARSING = "Error: Parsing failed"
ARGUMENT = "Exception in thread \"main\" java.lang.IllegalArgumentException"
STATE = "Exception in thread \"main\" java.lang.IllegalStateException"
NOTCOMPLETED = "Analysis not completed: there are still states to be processed."
CLASS = "Exception in thread \"main\" java.lang.ClassCastException"
UNSUPPORTED = "Exception in thread \"main\" de.uni_freiburg.informatik.ultimate.logic.SMTLIBException: Unsupported non-linear arithmetic"
REFINEMENT = "Refinement failed: Counterexample could not be ruled out and was found again"
ASSERTION = "Exception in thread \"main\" java.lang.AssertionError"

errors = [OPERATION,UNSOUND,PARSING,ARGUMENT,STATE,NOTCOMPLETED,CLASS,UNSUPPORTED,REFINEMENT,ASSERTION]

def inc(dic, val):
    if not val in dic:
        dic[val] = 1
    else:
        dic[val] += 1


In [3]:
def get_cpa_df(rootdir):
    df = {
        'TRUE':{},
        'FALSE':{},
        'UNKNOWN':{},
        'TIMEOUT':{},
        'ERROR':{},
    }
    _, dirs, _ = next(os.walk(rootdir))
    for dir in dirs:
        loopdir = "{}/{}".format(rootdir, dir)
        _, _, programs = next(os.walk(loopdir))
        for program in programs:
            if program[-4:] != ".out":
                continue

            with open("{}/{}".format(loopdir, program)) as f:
                str = f.read()
                if TRUE in str:
                    inc(df["TRUE"], dir)
                elif FALSE in str:
                    inc(df["FALSE"], dir)
                elif UNKNOWN in str:
                    inc(df["UNKNOWN"], dir)
                elif TIMEOUT in str:
                    inc(df["TIMEOUT"], dir)
                else:
                    inc(df["ERROR"], dir)
    return df

In [74]:
cpa_df = pd.DataFrame(get_cpa_df("cpa-checker-results")).fillna(0)
cpa_df = cpa_df.reindex(['TRUE', 'FALSE', 'UNKNOWN', 'TIMEOUT', 'ERROR'], axis=1)
cpa_df.sort_index(inplace=True)
print("CPACHECKER")
cpa_df

CPACHECKER


,TRUE,FALSE,UNKNOWN,TIMEOUT,ERROR
0,"62,022",21,7,18,79
1,"2,420",378,"1,444","1,514",1475
2,171,34,88,280,219
3,8,0,7,22,75
4,2,0,1,3,16
5,0,0,0,0,3
6,0,0,0,0,2


In [5]:
def extract(df, pos, cols, addts):
    df = df.iloc[[pos]][cols]
    for key, val in addts.items():
        df[key] = [val]
    return df

In [6]:
def get_ultimate_dfs(rootdir, csv_name, columns):
    dfs = []
    _, dirs, _ = next(os.walk(rootdir))
    for dir in dirs:
        loopdir = "{}/{}".format(rootdir, dir)
        _, programs, _ = next(os.walk(loopdir))
        for program in programs:
            filename = program.split(".c")[0] + ".c"
            program_dir = "{}/{}".format(loopdir, program)
            _, _, files = next(os.walk(program_dir))
            for file in files:
                if csv_name in file:
                    file_dir = "{}/{}".format(program_dir, file)
                    file_df = pd.read_csv(file_dir)
                    try:
                        dfs.append(extract(file_df, 5, columns, {'Loops': dir}))
                    except:
                        dfs.append(extract(file_df, 0, columns, {'Loops': dir}))
                    break
    return dfs

In [7]:
columns = ['Unnamed: 0', 'Inputfiles', 'AnalysisResult', 'Runtime (ns)', 'Peak memory consumption (bytes)']
ultimate_df = pd.concat(get_ultimate_dfs("ultimate-results", "Csv-Benchmark", columns))

In [73]:
df2 = ultimate_df
df2 = df2.groupby([df2.Loops, df2.AnalysisResult]).count().unstack().fillna(0).astype(int)['Unnamed: 0']
df2["ERROR"] = df2["ERROR"] + df2["SYNTAXERROR"] + df2["NORESULT"]
df2 = df2[['CORRECT', 'INCORRECT', 'UNPROVABLE', 'TIMEOUT', 'ERROR']]
df2.columns = ['TRUE', 'FALSE', 'UNKNOWN', 'TIMEOUT', 'ERROR']
df2 = pd.DataFrame(df2.to_dict())
print("ULTIMATE")
df2

ULTIMATE


,TRUE,FALSE,UNKNOWN,TIMEOUT,ERROR
0,58676,17,5,414,3034
1,4211,528,195,2014,283
2,107,35,16,587,47
3,1,0,1,107,3
4,0,2,0,20,0
5,0,0,0,3,0
6,0,0,0,2,0


In [75]:
print("TOTAL RESULTS")
pd.concat([df2, cpa_df], keys=["ULTIMATE", "CPACHECKER"])

TOTAL RESULTS


TRUE  FALSE  UNKNOWN  TIMEOUT  ERROR
ULTIMATE   0 58,676     17        5      414   3034
           1  4,211    528      195    2,014    283
           2    107     35       16      587     47
           3      1      0        1      107      3
           4      0      2        0       20      0
           5      0      0        0        3      0
           6      0      0        0        2      0
CPACHECKER 0 62,022     21        7       18     79
           1  2,420    378    1,444    1,514   1475
           2    171     34       88      280    219
           3      8      0        7       22     75
           4      2      0        1        3     16
           5      0      0        0        0      3
           6      0      0        0        0      2